In [1]:
import sys

sys.path.append("../..")

#Creation of the subsets and hazard rates

In [2]:
indiv_subsets = [frozenset([i]) for i in range(3)]
groups_subsets = [frozenset([0, 1]), frozenset([0, 1, 2])]

subsets = indiv_subsets+groups_subsets
print subsets

[frozenset([0]), frozenset([1]), frozenset([2]), frozenset([0, 1]), frozenset([0, 1, 2])]


In [3]:
indiv_lambdas = [0.0001*(200-i) for i in range(3)]
groups_lambdas = [2*0.001/(1+i) for i in range(2, 4)]

lambdas = indiv_lambdas+groups_lambdas
print lambdas

[0.02, 0.0199, 0.0198, 0.0006666666666666666, 0.0005]


In [4]:
from maths.copula.marshallolkin import StepWiseIntensitiesMarshallOlkinCopula

copula = StepWiseIntensitiesMarshallOlkinCopula(subsets, [lambdas], [1])

#print copula.intensities
#print
#print copula.subsets
#print
#print copula.pillars

In [5]:
print copula.get_indexes_including(0)
print copula.get_indexes_including(1)
print copula.get_indexes_including(2)

[0, 3, 4]
[1, 3, 4]
[2, 4]


In [13]:
copula.models[0] = None
print copula.models

[ <credit.default_models.StepwiseConstantIntensity object at 0x00000000045B87B8>


To generate the default times, we do the following:

\begin{equation*}
\eta_Y = \inf\{ t \geq 0, \int_0^t \gamma_s^Y ds > \xi_Y \}
\end{equation*}

The code uses the `brentq` routine from [scipy](http://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.optimize.brentq.html#scipy.optimize.brentq) to find the default time given the exponential random variable.

In [7]:
from scipy.stats import expon

exps = expon.rvs(size=[1, 3])
taus = copula.generate_default_times(0, exp_rvs=exps)

In [8]:
print -copula.models[0].log_survival_proba(taus[0, 0]), exps[0, 0]
print -copula.models[3].log_survival_proba(taus[0, 1]), exps[0, 1]
print -copula.models[4].log_survival_proba(taus[0, 2]), exps[0, 2]

1.08265826222 1.08265826222
0.520988339215 0.520988339215
1.35047189419 1.35047189419


In [9]:
print copula.generate_default_times(1, 10)

[[  2.46331018e-01   7.57451309e+03   2.58521038e+02]
 [  1.46994441e+01   2.50040072e+03   3.95151117e+03]
 [  1.16934383e+02   1.38208201e+03   3.45974533e+03]
 [  1.19707051e+02   2.01233706e+02   2.54691377e+02]
 [  5.19767013e+01   2.04605387e+02   8.08256862e+00]
 [  1.30552381e+02   1.55909856e+02   7.53646078e+03]
 [  1.06594855e+02   1.45432321e+03   4.48316332e+02]
 [  1.19201988e+01   9.75555330e+02   2.11209032e+03]
 [  1.85121214e+00   1.21975062e+03   5.79444509e+02]
 [  3.94286255e+01   4.65606133e+01   1.71957584e+03]]


In [10]:
print copula.generate_default_times(2, 10)

[[  4.32776800e+00   9.54190932e+02]
 [  2.45064639e+01   7.88949914e+02]
 [  1.39486227e+01   3.27540458e+03]
 [  2.28696921e+02   3.99785864e+03]
 [  5.32264922e+00   6.29548283e+02]
 [  7.15920194e+01   1.13273862e+03]
 [  5.01665420e+00   3.02540595e+03]
 [  3.54888753e+01   3.54491331e+02]
 [  7.08715619e-01   2.37834246e+03]
 [  7.70016845e+01   3.23012844e+03]]
